# 😊 Análisis de Emociones en Descripciones de Libros

## Objetivo

Este notebook realiza un **análisis de emociones** en las descripciones de libros usando **modelos de Deep Learning pre-entrenados**. 

### Características principales:
- **Detección de emociones**: Identifica 7 emociones diferentes en cada descripción
- **Análisis por oración**: Divide descripciones en oraciones para análisis detallado
- **Scoring**: Calcula puntuaciones de confianza para cada emoción
- **Agregación**: Obtiene la máxima emoción detectada por libro
- **Enriquecimiento de datos**: Agrega columnas de emociones al dataset
- **Dataset final**: Genera `books_with_emotions.csv` con análisis emocional completo

In [1]:
# ========================
# PASO 1: Cargar los datos
# ========================
import pandas as pd

# Cargar el dataset de libros con categorías (resultado del notebook anterior)
books = pd.read_csv("../data/books_with_categories.csv")

## 1️⃣ Cargar Dataset

**¿Qué hace?**: Importa el archivo CSV con los libros clasificados del notebook anterior.

**Archivo**: `books_with_categories.csv`  
**Contenido**: ~2000 libros con categorías, descripciones y metadatos

In [2]:
# ========================
# PASO 2: Cargar modelo de análisis de emociones
# ========================
from transformers import pipeline

# Cargar el clasificador de emociones
# Modelo: j-hartmann/emotion-english-distilroberta-base
# Detecta: anger, disgust, fear, joy, sadness, surprise, neutral
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,  # Retorna puntuaciones para todas las emociones
                      device=-1)  # GPU en Mac (cambiar a "cuda" en NVIDIA o "cpu")

# Prueba del modelo con un ejemplo simple
classifier("I love this!")

[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528688922524452},
  {'label': 'neutral', 'score': 0.005764589179307222},
  {'label': 'anger', 'score': 0.004419790115207434},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.001611991785466671},
  {'label': 'fear', 'score': 0.0004138524236623198}]]

## 2️⃣ Cargar Modelo de Análisis de Emociones

### ¿Qué es este modelo?

**DistilRoBERTa** es un modelo de IA pre-entrenado que detecta 7 emociones diferentes en texto en inglés:

| Emoción | Descripción |
|---------|-------------|
| 😠 **Anger** | Rabia, enojo |
| 🤢 **Disgust** | Asco, repugnancia |
| 😨 **Fear** | Miedo, temor |
| 😊 **Joy** | Alegría, felicidad |
| 😢 **Sadness** | Tristeza, melancolía |
| 😮 **Surprise** | Sorpresa, asombro |
| 😐 **Neutral** | Neutral, sin emociones claras |

### Características del modelo:
- 🚀 Basado en BERT (modelo transformer)
- ⚡ Optimizado para velocidad con DistilBERT
- 📊 Retorna puntuaciones de confianza (0-1) para cada emoción
- 🎯 Funciona bien con descripciones de libros

In [3]:
# Obtener la descripción del primer libro para prueba
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [4]:
# Clasificar la descripción completa del libro
# Retorna una lista con puntuaciones para todas las emociones
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548399925231934},
  {'label': 'neutral', 'score': 0.1698525995016098},
  {'label': 'sadness', 'score': 0.11640939861536026},
  {'label': 'surprise', 'score': 0.02070068009197712},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161462128162384},
  {'label': 'anger', 'score': 0.003935154061764479}]]

In [5]:
# Dividir la descripción en oraciones y clasificar cada una
# Esto proporciona un análisis más detallado (la descripción es demasiado larga)
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.729602038860321},
  {'label': 'neutral', 'score': 0.14038598537445068},
  {'label': 'fear', 'score': 0.06816231459379196},
  {'label': 'joy', 'score': 0.0479426383972168},
  {'label': 'anger', 'score': 0.009156374260783195},
  {'label': 'disgust', 'score': 0.002628476358950138},
  {'label': 'sadness', 'score': 0.002122163539752364}],
 [{'label': 'neutral', 'score': 0.44937053322792053},
  {'label': 'disgust', 'score': 0.2735915184020996},
  {'label': 'joy', 'score': 0.10908309370279312},
  {'label': 'sadness', 'score': 0.0936274453997612},
  {'label': 'anger', 'score': 0.04047830402851105},
  {'label': 'surprise', 'score': 0.02697017788887024},
  {'label': 'fear', 'score': 0.006879044696688652}],
 [{'label': 'neutral', 'score': 0.6462159156799316},
  {'label': 'sadness', 'score': 0.242733433842659},
  {'label': 'disgust', 'score': 0.043422672897577286},
  {'label': 'surprise', 'score': 0.02830057218670845},
  {'label': 'joy', 'score': 0.01421149075031

## 3️⃣ Pruebas de Análisis Emocional

**¿Qué hace?**: 
1. Analiza la descripción completa de un libro
2. Divide la descripción en oraciones (separadas por puntos)
3. Analiza emociones en cada oración por separado

**Razón**: Las descripciones largas son mejor analizadas por partes, obteniendo un análisis emocional más rico y matizado

In [6]:
# Preparar las predicciones para análisis
# Dividir en oraciones y clasificar
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [7]:
# Ver la primera oración de la descripción
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [8]:
# Ver las predicciones emocionales para la primera oración
predictions[0]

[{'label': 'surprise', 'score': 0.729602038860321},
 {'label': 'neutral', 'score': 0.14038598537445068},
 {'label': 'fear', 'score': 0.06816231459379196},
 {'label': 'joy', 'score': 0.0479426383972168},
 {'label': 'anger', 'score': 0.009156374260783195},
 {'label': 'disgust', 'score': 0.002628476358950138},
 {'label': 'sadness', 'score': 0.002122163539752364}]

In [9]:
# Ver una oración diferente (la cuarta)
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [10]:
# Ver las predicciones emocionales para esa oración
predictions[3]

[{'label': 'fear', 'score': 0.9281681180000305},
 {'label': 'anger', 'score': 0.03219091519713402},
 {'label': 'neutral', 'score': 0.012808694504201412},
 {'label': 'sadness', 'score': 0.008756881579756737},
 {'label': 'surprise', 'score': 0.008597923442721367},
 {'label': 'disgust', 'score': 0.008431830443441868},
 {'label': 'joy', 'score': 0.001045584212988615}]

In [11]:
# Ver todas las predicciones
predictions

[[{'label': 'surprise', 'score': 0.729602038860321},
  {'label': 'neutral', 'score': 0.14038598537445068},
  {'label': 'fear', 'score': 0.06816231459379196},
  {'label': 'joy', 'score': 0.0479426383972168},
  {'label': 'anger', 'score': 0.009156374260783195},
  {'label': 'disgust', 'score': 0.002628476358950138},
  {'label': 'sadness', 'score': 0.002122163539752364}],
 [{'label': 'neutral', 'score': 0.44937053322792053},
  {'label': 'disgust', 'score': 0.2735915184020996},
  {'label': 'joy', 'score': 0.10908309370279312},
  {'label': 'sadness', 'score': 0.0936274453997612},
  {'label': 'anger', 'score': 0.04047830402851105},
  {'label': 'surprise', 'score': 0.02697017788887024},
  {'label': 'fear', 'score': 0.006879044696688652}],
 [{'label': 'neutral', 'score': 0.6462159156799316},
  {'label': 'sadness', 'score': 0.242733433842659},
  {'label': 'disgust', 'score': 0.043422672897577286},
  {'label': 'surprise', 'score': 0.02830057218670845},
  {'label': 'joy', 'score': 0.01421149075031

In [12]:
# Ordenar las predicciones de la primera oración por etiqueta
# Esto permite ver todas las emociones en orden
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156374260783195},
 {'label': 'disgust', 'score': 0.002628476358950138},
 {'label': 'fear', 'score': 0.06816231459379196},
 {'label': 'joy', 'score': 0.0479426383972168},
 {'label': 'neutral', 'score': 0.14038598537445068},
 {'label': 'sadness', 'score': 0.002122163539752364},
 {'label': 'surprise', 'score': 0.729602038860321}]

In [13]:
# ========================
# PASO 3: Crear función de agregación de emociones
# ========================
import numpy as np

# Definir las 7 emociones que detecta el modelo
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]

# Listas para almacenar resultados
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    """
    Calcula la puntuación máxima para cada emoción en todas las oraciones.
    
    Args:
        predictions: Lista de predicciones (una por oración)
        
    Returns:
        Diccionario con la puntuación máxima para cada emoción
    """
    # Crear diccionario para almacenar puntuaciones por emoción
    per_emotion_scores = {label: [] for label in emotion_labels}
    
    # Para cada predicción (oración)
    for prediction in predictions:
        # Ordenar por nombre de etiqueta para mantener consistencia
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        
        # Para cada emoción en orden
        for index, label in enumerate(emotion_labels):
            # Guardar la puntuación de esa emoción
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    
    # Retornar el máximo de cada emoción (la más prevalente en la descripción)
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

## 4️⃣ Crear Función de Agregación de Emociones

**¿Qué hace?**: 
1. Toma todas las predicciones emocionales de una descripción (múltiples oraciones)
2. Extrae la puntuación máxima para cada emoción
3. Retorna un diccionario con emociones y sus puntuaciones máximas

**Lógica**:
```
Descripción → Dividir en oraciones → Clasificar cada oración → 
Tomar máximo de cada emoción → Resultado final por libro
```

**Resultado**: Un diccionario con 7 emociones y sus puntuaciones (0-1) para cada libro

In [14]:
# Prueba con los primeros 10 libros
for i in range(10):
    # Guardar ISBN del libro
    isbn.append(books["isbn13"][i])
    
    # Dividir descripción en oraciones
    sentences = books["description"][i].split(".")
    
    # Clasificar cada oración
    predictions = classifier(sentences)
    
    # Obtener puntuaciones máximas para cada emoción
    max_scores = calculate_max_emotion_scores(predictions)
    
    # Guardar puntuaciones en el diccionario
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

### Prueba con 10 Libros

**¿Qué hace?**: Ejecuta el análisis emocional en los primeros 10 libros del dataset.

**Propósito**: Verificar que todo funciona correctamente antes de ejecutar en todo el dataset

In [15]:
# Ver los resultados de la prueba
emotion_scores

{'anger': [0.0641336515545845,
  0.6126194000244141,
  0.0641336515545845,
  0.3514845371246338,
  0.0814124271273613,
  0.23222433030605316,
  0.5381841063499451,
  0.0641336515545845,
  0.3006698489189148,
  0.0641336515545845],
 'disgust': [0.2735915184020996,
  0.34828439354896545,
  0.10400670766830444,
  0.1507224291563034,
  0.18449550867080688,
  0.7271752953529358,
  0.15585480630397797,
  0.10400670766830444,
  0.27948102355003357,
  0.1779264211654663],
 'fear': [0.9281681180000305,
  0.9425276517868042,
  0.9723207950592041,
  0.3607056736946106,
  0.09504345804452896,
  0.05136283487081528,
  0.7474274635314941,
  0.404496967792511,
  0.9155239462852478,
  0.05136283487081528],
 'joy': [0.9327981472015381,
  0.7044221758842468,
  0.7672379016876221,
  0.25188156962394714,
  0.04056442901492119,
  0.04337584972381592,
  0.8725655674934387,
  0.04056442901492119,
  0.04056442901492119,
  0.04056442901492119],
 'sadness': [0.6462159156799316,
  0.8879393339157104,
  0.5494766

In [16]:
# ========================
# PASO 4: Analizar TODOS los libros
# ========================
from tqdm import tqdm

# Re-inicializar variables para análisis completo
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

# Iterar sobre todos los libros del dataset
for i in tqdm(range(len(books))):
    # Guardar ISBN
    isbn.append(books["isbn13"][i])
    
    # Dividir descripción en oraciones
    sentences = books["description"][i].split(".")
    
    # Clasificar emociones
    predictions = classifier(sentences)
    
    # Calcular máximos
    max_scores = calculate_max_emotion_scores(predictions)
    
    # Guardar puntuaciones
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [11:27<00:00,  7.56it/s]


## 5️⃣ Análisis Completo de Todos los Libros

**¿Qué hace?**: 
1. Itera sobre TODOS los libros del dataset (~2000)
2. Para cada libro: divide en oraciones y clasifica emociones
3. Extrae las puntuaciones máximas de cada emoción
4. Guarda los resultados con barra de progreso

**Nota**: Usa `tqdm` para mostrar progreso (esto puede tomar varios minutos)

**Complejidad**: O(n × m) donde n = libros, m = oraciones por descripción

In [17]:
# ========================
# PASO 5: Crear DataFrame de emociones
# ========================
# Crear DataFrame con las puntuaciones de emociones
emotions_df = pd.DataFrame(emotion_scores)

# Agregar la columna ISBN para poder hacer merge después
emotions_df["isbn13"] = isbn

### Crear DataFrame de Emociones

**¿Qué hace?**:
1. Convierte el diccionario `emotion_scores` en un DataFrame
2. Agrega la columna ISBN13 para facilitar el merge posterior

**Estructura del DataFrame**:
- 7 columnas de emociones (anger, disgust, fear, joy, sadness, surprise, neutral)
- 1 columna isbn13
- Una fila por cada libro (~2000 filas)

In [18]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765,9780006178736
3,0.351485,0.150722,0.360706,0.251882,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255171,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375754,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759456,0.951104,0.368110,0.078765,9788185300535


In [19]:
# ========================
# PASO 6: Integrar emociones con dataset principal
# ========================
# Hacer merge del DataFrame de emociones con el dataset de libros
# Usar ISBN13 como clave de union
books = pd.merge(books, emotions_df, on="isbn13")

### Integrar Emociones con Dataset Principal

**¿Qué hace?**: 
1. Combina el DataFrame de emociones con el dataset original de libros
2. Usa ISBN13 como clave de union (inner join)
3. Cada libro ahora tiene sus 7 puntuaciones emocionales

**Resultado**: Dataset enriquecido con información emocional

In [20]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252546
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351485,0.150722,0.360706,0.251882,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148209,0.030643,0.919165,0.255171,0.853721,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375754,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759456,0.951104,0.368110,0.078765


In [23]:
# ========================
# PASO 7: Guardar dataset final
# ========================
# Guardar el dataset con análisis emocional completo
books.to_csv("../data/books_with_emotions.csv", index=False)

print("✅ Dataset con emociones guardado como 'books_with_emotions.csv'")

✅ Dataset con emociones guardado como 'books_with_emotions.csv'


## 6️⃣ Guardar Dataset Final

**¿Qué hace?**: Exporta el dataset completo con análisis emocional.

**Archivo de salida**: `books_with_emotions.csv`

**Contenido**: Todos los ~2000 libros con:
- ✅ Todas las columnas originales (título, autor, descripción, etc.)
- ✅ Categorías simples
- ✅ **7 nuevas columnas de emociones** (anger, disgust, fear, joy, sadness, surprise, neutral)
- ✅ Puntuaciones de emociones (0-1) para cada libro

---

## 📊 Resumen del Pipeline de Análisis de Emociones

### Arquitectura del Sistema

```
┌──────────────────────────────────┐
│ books_with_categories.csv        │
│ (~2000 libros con categorías)    │
└─────────────┬────────────────────┘
              │
              ▼
┌──────────────────────────────────┐
│ Cargar modelo DistilRoBERTa      │
│ (7 emociones preentrenadas)      │
└─────────────┬────────────────────┘
              │
              ▼
┌──────────────────────────────────┐
│ Para cada libro:                 │
│ 1. Dividir descripción en oraciones
│ 2. Clasificar emociones de c/oración
│ 3. Extraer máximos por emoción   │
└─────────────┬────────────────────┘
              │
              ▼
┌──────────────────────────────────┐
│ Crear DataFrame de emociones     │
│ (7 columnas + ISBN)              │
└─────────────┬────────────────────┘
              │
              ▼
┌──────────────────────────────────┐
│ Merge con dataset original       │
│ (usando ISBN13 como clave)       │
└─────────────┬────────────────────┘
              │
              ▼
┌──────────────────────────────────┐
│ books_with_emotions.csv (FINAL)  │
│ ✅ Dataset completo con emociones │
└──────────────────────────────────┘
```

### Flujo Paso a Paso

| Paso | Actividad | Descripción |
|------|-----------|-------------|
| 1️⃣ | Cargar datos | Importar `books_with_categories.csv` |
| 2️⃣ | Cargar modelo | DistilRoBERTa para detectar 7 emociones |
| 3️⃣ | Pruebas | Verificar funcionamiento con ejemplos |
| 4️⃣ | Función agregación | Crear `calculate_max_emotion_scores()` |
| 5️⃣ | Análisis completo | Procesar todos los ~2000 libros |
| 6️⃣ | Crear DataFrame | Convertir resultados en tabla |
| 7️⃣ | Integrar | Merge con dataset original |
| 8️⃣ | Guardar | Exportar `books_with_emotions.csv` |

### Las 7 Emociones Detectadas

| Emoción | Emoji | Ejemplos de contenido |
|---------|-------|----------------------|
| **Anger** | 😠 | "Furioso", "Indignado", "Ira destructiva" |
| **Disgust** | 🤢 | "Repugnante", "Asqueroso", "Repulsivo" |
| **Fear** | 😨 | "Aterrador", "Miedo", "Pánico" |
| **Joy** | 😊 | "Alegría", "Felicidad", "Esperanza" |
| **Sadness** | 😢 | "Triste", "Melancolía", "Desolador" |
| **Surprise** | 😮 | "Sorpresa", "Inesperado", "Giro argumental" |
| **Neutral** | 😐 | Texto factual sin emociones claras |

### Casos de Uso

Los análisis emocionales pueden usarse para:

🎯 **Sistema de Recomendación**:
- Recomendar libros con emociones similares a preferencias del usuario

📊 **Análisis de Catálogo**:
- Identificar libros más "optimistas" vs "melancólicos"

👤 **Perfilamiento**:
- Crear perfiles emocionales de géneros

🔍 **Búsqueda Avanzada**:
- "Buscar libros alegres" / "Buscar libros aterradores"

### Tecnologías Utilizadas

- **Transformers**: Acceso a modelos pre-entrenados de HuggingFace
- **DistilRoBERTa**: Modelo base para clasificación emocional
- **Pandas**: Manipulación de datos
- **Numpy**: Operaciones numéricas
- **tqdm**: Barras de progreso para loops largos